# MeMo Torch Initial MEMORIZATION EXPERIMENTS
Version integrated with Torch - NN

In [ ]:
import torch
from MeMoPyTorch.modelling_memo import MeMo
from MeMoPyTorch.modelling_memo_tokenizer import MeMoTokenizer
from MeMoPyTorch.evaluating_memo import Evaluation

import numpy as np
import pandas as pd

Memo Experiment Function

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
    device = 'cuda'

def MemoExperiment(dim=1024,heads=4,layers=4, chunk_length=1024, decoys=20):
    d,h,l = dim, heads, layers
    max_length = chunk_length 
    samples = 30
    iterations = 10
    e = Evaluation()

    results = pd.DataFrame()
    tokenizer = MeMoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b", 
                                              padding_side='left', truncation_side='left', 
                                              max_length=max_length, head_number=h)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.pad_token_id
    # Intializing Memo 
    model = MeMo(inner_dim=d, 
                 num_of_heads=h, 
                 num_of_layers=l, 
                 chunk_length=max_length, 
                 num_embeddings=tokenizer.vocab_size, 
                 padding_idx=tokenizer.pad_token_id, 
                 device=device)
    
    for i in range(iterations):
        input_ids = torch.tensor(np.random.randint(0, tokenizer.vocab_size, size=(samples, chunk_length+1)))
        #step = samples*(chunk_length+1)
        #input_ids = torch.tensor(np.array([k+1 for k in range(i*step,(i+1)*step)])).reshape((samples,chunk_length+1))
        ##### Replicate Sequences of tokens
        for j in np.random.randint(5+h, chunk_length - h, decoys):
            input_ids[:,j:j+h]=input_ids[:,5:5+h]
    
        memo_input = {'input_ids': input_ids[..., :-1], 'labels': input_ids[..., 1:]}
        #print(memo_input)
        if i == 0 :
            initial_memo_input = memo_input
        model.memorize_text(memo_input)
        e1 = e.check_pretokenized(model, tokenizer, initial_memo_input['input_ids'], starting_point=h)
        print("Memorization level of first text  : ", e1) 
        e2 = e.check_pretokenized(model, tokenizer, memo_input['input_ids'], starting_point=h)
        print("Memorization level of first text  : ", e2)
        results = pd.concat([results,pd.DataFrame([{"dim": dim, "heads": heads, "layers": layers, 
                                                    "chunk_length": chunk_length, "decoys": decoys, "initial":e1.item(), "current":e2.item(), "iteration":i, "samples":samples  }])])
    return results

In [ ]:
results = pd.DataFrame()
#results = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")

#for dim in [1024,2048]:
for dim in [1024,2048,4096]:
    for layers in [1,2,3]: 
        curr_res = MemoExperiment(dim=dim,heads=4,layers=layers, chunk_length=1024, decoys=0)
        results = pd.concat([results,curr_res])
        results.to_excel("Experiments/MemorizationResults-MEMO.xlsx")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")

#{"dim": dim, "heads": heads, "layers": layers, "chunk_length": chunk_length, "decoys": decoys, "initial":e1, "current":e2, "iteration":i  }
heads = 4
decoys = 20
df["eval"] = (df["initial"] + df["current"])/2
df["sequences"] = df["iteration"]*df["samples"]*df["chunk_length"]
#dim = 4096
dim = 8192
for layers in [1,2,3]:
    plt.plot("sequences","eval",data=df.loc[(df["layers"]==layers)&(df["dim"]==dim)&(df["decoys"]==decoys)][["sequences","eval"]], label=f"layers = ${layers}$",marker="o",linewidth=1)


plt.ylabel("Memorization")
plt.xlabel("Sequences")
plt.title(f"Heads:{heads} Decoys:{decoys} $d$:{dim}")
plt.legend()
#plt.show()
plt.savefig(f"Experiments/MEMOstoring_capacity_vs_layers_Heads_{heads}_Decoys_{decoys}_d_{dim}.png",dpi=300)



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#{"dim": dim, "heads": heads, "layers": layers, "chunk_length": chunk_length, "decoys": decoys, "initial":e1, "current":e2, "iteration":i  }

heads = 4
decoys = 20

df = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")
df["eval"] = (df["initial"] + df["current"])/2
df["sequences"] = df["iteration"]*df["samples"]*df["chunk_length"]
layers=3
for dim in [1024,2048,4096,8192]:
    plt.plot("sequences","eval",data=df.loc[(df["layers"]==layers)&(df["dim"]==dim)&(df["decoys"]==decoys)][["sequences","eval"]], label=f"dim = ${dim}$",marker="o",linewidth=1)


plt.ylabel("Memorization")
plt.xlabel("Sequences")
plt.title(f"Heads:{heads} Decoys:{decoys} $layers$:{layers}")
plt.legend()
#plt.show()
plt.savefig(f"Experiments/MEMOstoring_capacity_vs_dim_Heads_{heads}_Decoys_{decoys}_layers_{layers}.png",dpi=300)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#{"dim": dim, "heads": heads, "layers": layers, "chunk_length": chunk_length, "decoys": decoys, "initial":e1, "current":e2, "iteration":i  }

heads = 4
decoys = 40

df = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")
df["eval"] = (df["initial"] + df["current"])/2
df["sequences"] = df["iteration"]*df["samples"]*df["chunk_length"]
layers=3
for dim in [1024,2048,4096,8192]:
    plt.plot("sequences","eval",data=df.loc[(df["layers"]==layers)&(df["dim"]==dim)&(df["decoys"]==decoys)][["sequences","eval"]], label=f"dim = ${dim}$",marker="o",linewidth=1)


plt.ylabel("Memorization")
plt.xlabel("Sequences")
plt.title(f"Heads:{heads} Decoys:{decoys} $layers$:{layers}")
plt.legend()
#plt.show()
plt.savefig(f"Experiments/MEMOstoring_capacity_vs_dim_Heads_{heads}_Decoys_{decoys}_layers_{layers}.png",dpi=300)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#{"dim": dim, "heads": heads, "layers": layers, "chunk_length": chunk_length, "decoys": decoys, "initial":e1, "current":e2, "iteration":i  }

heads = 4
decoys = 40
df = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")
df["eval"] = (df["initial"] + df["current"])/2
df["sequences"] = df["iteration"]*df["samples"]*df["chunk_length"]
dim = 8192
for layers in [1,2,3]:
    plt.plot("sequences","eval",data=df.loc[(df["layers"]==layers)&(df["dim"]==dim)&(df["decoys"]==decoys)][["sequences","eval"]], label=f"layers:${layers}$",marker="o",linewidth=1)


plt.ylabel("Memorization")
plt.xlabel("Sequences")
plt.title(f"Heads:{heads} Decoys:{decoys} $d$:{dim}")
plt.legend()
#plt.show()
plt.savefig(f"Experiments/MEMOstoring_capacity_vs_layers_Heads_{heads}_Decoys_{decoys}_d_{dim}.png",dpi=300)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")

#{"dim": dim, "heads": heads, "layers": layers, "chunk_length": chunk_length, "decoys": decoys, "initial":e1, "current":e2, "iteration":i  }
heads = 4
decoys = 0
df["eval"] = (df["initial"] + df["current"])/2
df["sequences"] = df["iteration"]*df["samples"]*df["chunk_length"]
dim = 4096
for layers in [1,2,3]:
    plt.plot("sequences","eval",data=df.loc[(df["layers"]==layers)&(df["dim"]==dim)&(df["decoys"]==decoys)][["sequences","eval"]], label=f"layers = ${layers}$",marker="o",linewidth=1)


plt.ylabel("Memorization")
plt.xlabel("Sequences")
plt.title(f"Heads:{heads} Decoys:{decoys} $d$:{dim}")
plt.legend()
#plt.show()
plt.savefig(f"Experiments/MEMOstoring_capacity_vs_layers_Heads_{heads}_Decoys_{decoys}_d_{dim}.png",dpi=300)



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#{"dim": dim, "heads": heads, "layers": layers, "chunk_length": chunk_length, "decoys": decoys, "initial":e1, "current":e2, "iteration":i  }

heads = 4
decoys = 0

df = pd.read_excel("Experiments/MemorizationResults-MEMO.xlsx")
df["eval"] = (df["initial"] + df["current"])/2
df["sequences"] = df["iteration"]*df["samples"]*df["chunk_length"]
layers=3
for dim in [1024,2048,4096]:
    plt.plot("sequences","eval",data=df.loc[(df["layers"]==layers)&(df["dim"]==dim)&(df["decoys"]==decoys)][["sequences","eval"]], label=f"dim = ${dim}$",marker="o",linewidth=1)


plt.ylabel("Memorization")
plt.xlabel("Sequences")
plt.title(f"Heads:{heads} Decoys:{decoys} $layers$:{layers}")
plt.legend()
#plt.show()
plt.savefig(f"Experiments/MEMOstoring_capacity_vs_dim_Heads_{heads}_Decoys_{decoys}_layers_{layers}.png",dpi=300)


In [ ]:
!nvidia-smi